In [1]:
using Pkg
Pkg.add("HDF5")
Pkg.add("FITSIO")
Pkg.add("GZip")
Pkg.add("Plots")
Pkg.add("Statistics")

   Resolving package versions...
  No Changes to `C:\Users\asus4\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\asus4\.julia\environments\v1.11\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\asus4\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\asus4\.julia\environments\v1.11\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\asus4\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\asus4\.julia\environments\v1.11\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\asus4\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\asus4\.julia\environments\v1.11\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\asus4\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\asus4\.julia\environments\v1.11\Manifest.toml`


In [3]:
using Pkg
Pkg.add("Downloads")

   Resolving package versions...
  No Changes to `C:\Users\asus4\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\asus4\.julia\environments\v1.11\Manifest.toml`


In [4]:
using Downloads

url = "https://archive.stsci.edu/pub/kepler/lightcurves/0114/011446443/kplr011446443-2009131110544_slc.fits"
download_path = "kplr011446443.fits"
Downloads.download(url, download_path)
println("Downloaded to: $download_path")

Downloaded to: kplr011446443.fits


In [22]:
using FITSIO
struct Meta
    headers::Vector{Dict{String,Any}}
end
struct EventList{T}
    filename::String
    times::Vector{T}
    energies::Vector{T}
    metadata::Meta
end
function readevents(path; T=Float64)
    headers = Dict{String,Any}[]
    times = T[]
    energies = T[]
    
    FITS(path, "r") do f
        for i in 1:length(f)  # Iterate over HDUs
            hdu = f[i]
            
            # Extract header information as a dictionary
            header_dict = Dict{String,Any}()
            for key in keys(read_header(hdu))
                header_dict[string(key)] = read_header(hdu)[key]
            end
            push!(headers, header_dict)
            
            # Check if the HDU is a table
            if isa(hdu, TableHDU)
                # Get column names using the correct FITSIO method
                colnames = FITSIO.colnames(hdu)
                
                if "TIME" in colnames
                    times = convert(Vector{T}, read(hdu, "TIME"))
                end
                if "ENERGY" in colnames
                    energies = convert(Vector{T}, read(hdu, "ENERGY"))
                end
            end
        end
    end
    
    metadata = Meta(headers)
    return EventList{T}(path, times, energies, metadata)
end


readevents (generic function with 1 method)

In [24]:
filename = "kplr011446443.fits"
event_data = readevents(filename, T=Float32)
println("Filename: ", event_data.filename)
println("Number of HDUs: ", length(event_data.metadata.headers))
println("Number of time values: ", length(event_data.times))
println("Number of energy values: ", length(event_data.energies))
println("Data Type T: ", eltype(event_data.times))

Filename: kplr011446443.fits
Number of HDUs: 3
Number of time values: 14280
Number of energy values: 0
Data Type T: Float32


In [18]:
using FITSIO
using Test

@testset "EventList Tests" begin
    # Test 1: Create a sample FITS file for testing
    @testset "Sample FITS file creation" begin
        test_dir = mktempdir()
        sample_file = joinpath(test_dir, "sample.fits")
        f = FITS(sample_file, "w")
        write(f, Int[])  # Empty primary array       
        # Create a binary table HDU with TIME and ENERGY columns
        times = Float64[1.0, 2.0, 3.0, 4.0, 5.0]
        energies = Float64[10.0, 20.0, 15.0, 25.0, 30.0]       
        # Add a binary table extension
        table = Dict{String, Array}()
        table["TIME"] = times
        table["ENERGY"] = energies
        write(f, table)
        close(f)
        
        @test isfile(sample_file)
        
        # Test reading the sample file
        data = readevents(sample_file)
        @test data.filename == sample_file
        @test length(data.times) == 5
        @test length(data.energies) == 5
        @test eltype(data.times) == Float64
        @test eltype(data.energies) == Float64
    end
    
    # Test 2: Test with different data types
    @testset "Different data types" begin
        test_dir = mktempdir()
        sample_file = joinpath(test_dir, "sample_float32.fits")
        f = FITS(sample_file, "w")
        write(f, Int[]) 
        # Create data
        times = Float64[1.0, 2.0, 3.0]
        energies = Float64[10.0, 20.0, 30.0]
        table = Dict{String, Array}()
        table["TIME"] = times
        table["ENERGY"] = energies
        write(f, table)
        close(f)
        # Test with Float32
        data_f32 = readevents(sample_file, T=Float32)
        @test eltype(data_f32.times) == Float32
        @test eltype(data_f32.energies) == Float32
        # Test with Int64
        data_i64 = readevents(sample_file, T=Int64)
        @test eltype(data_i64.times) == Int64
        @test eltype(data_i64.energies) == Int64
    end
    
    # Test 3: Test with missing columns
    @testset "Missing columns" begin
        test_dir = mktempdir()
        sample_file = joinpath(test_dir, "sample_no_energy.fits")   
        # Create a sample FITS file with only TIME column
        f = FITS(sample_file, "w")
        write(f, Int[]) 
        times = Float64[1.0, 2.0, 3.0]
        table = Dict{String, Array}()
        table["TIME"] = times
        write(f, table)
        close(f)
        data = readevents(sample_file)
        @test length(data.times) == 3
        @test length(data.energies) == 0        
        #create a file with only ENERGY column
        sample_file2 = joinpath(test_dir, "sample_no_time.fits")
        f = FITS(sample_file2, "w")
        write(f, Int[])  # Empty primary array
        energies = Float64[10.0, 20.0, 30.0]
        table = Dict{String, Array}()
        table["ENERGY"] = energies
        write(f, table)
        close(f)
        data2 = readevents(sample_file2)
        @test length(data2.times) == 0  # No TIME column
        @test length(data2.energies) == 3
    end
    
    # Test 4: Test with multiple HDUs
    @testset "Multiple HDUs" begin
        test_dir = mktempdir()
        sample_file = joinpath(test_dir, "sample_multi_hdu.fits")      
        # Create a sample FITS file with multiple HDUs
        f = FITS(sample_file, "w")
        write(f, Int[]) 
        times1 = Float64[1.0, 2.0, 3.0]
        energies1 = Float64[10.0, 20.0, 30.0]
        table1 = Dict{String, Array}()
        table1["TIME"] = times1
        table1["ENERGY"] = energies1
        write(f, table1)
        # Second table HDU (with OTHER column)
        other_data = Float64[100.0, 200.0, 300.0]
        table2 = Dict{String, Array}()
        table2["OTHER"] = other_data
        write(f, table2)
        # Third table HDU (with TIME only)
        times3 = Float64[4.0, 5.0, 6.0]
        table3 = Dict{String, Array}()
        table3["TIME"] = times3
        write(f, table3) 
        close(f)
        data = readevents(sample_file)
        @test length(data.metadata.headers) == 4  # Primary + 3 table HDUs
        # Should read the first HDU with both TIME and ENERGY
        @test length(data.times) == 3
        @test length(data.energies) == 3
    end
    
    # Test 5: Test with real test files if available
    @testset "Real test files" begin
        if isfile("monol_testA.evt")
            @testset "monol_testA.evt" begin
                data = readevents("monol_testA.evt")
                @test data.filename == "monol_testA.evt"
                @test length(data.metadata.headers) > 0
            end
        else
            @info "Test file 'monol_testA.evt' not found. Skipping this test."
        end
    end
    
    @testset "Error handling" begin
        # Test with non-existent file - using a more generic approach
        @test_throws Exception readevents("non_existent_file.fits")
        
        # Test with invalid FITS file
        invalid_file = tempname()
        open(invalid_file, "w") do io
            write(io, "This is not a FITS file")
        end
        @test_throws Exception readevents(invalid_file)
    end
end

Test Summary:   | Pass  Total  Time
EventList Tests |   21     21  0.1s


Test.DefaultTestSet("EventList Tests", Any[Test.DefaultTestSet("Sample FITS file creation", Any[], 6, false, false, true, 1.742618368561e9, 1.742618368588e9, false, "In[18]"), Test.DefaultTestSet("Different data types", Any[], 4, false, false, true, 1.742618368588e9, 1.742618368609e9, false, "In[18]"), Test.DefaultTestSet("Missing columns", Any[], 4, false, false, true, 1.742618368609e9, 1.742618368641e9, false, "In[18]"), Test.DefaultTestSet("Multiple HDUs", Any[], 3, false, false, true, 1.742618368641e9, 1.742618368663e9, false, "In[18]"), Test.DefaultTestSet("Real test files", Any[Test.DefaultTestSet("monol_testA.evt", Any[], 2, false, false, true, 1.742618368664e9, 1.742618368664e9, false, "In[18]")], 0, false, false, true, 1.742618368664e9, 1.742618368664e9, false, "In[18]"), Test.DefaultTestSet("Error handling", Any[], 2, false, false, true, 1.742618368664e9, 1.742618368679e9, false, "In[18]")], 0, false, false, true, 1.742618368561e9, 1.742618368679e9, false, "In[18]")